In [2]:
import networkx as nx
import numpy as np
import csv

# Load data set into graph
graph = nx.DiGraph()

with open("Iron_dealers_data.csv") as f:
    reader = csv.reader(f)
    next(reader) # skip header
    for row in reader:
        seller_id, buyer_id, value = row
#         if edge is already present then add weights to it(instead of adding duplicate edges)
        if(graph.has_edge(seller_id, buyer_id)):
            graph[seller_id][buyer_id]['weight'] += float(value)
        else:
            graph.add_edge(seller_id, buyer_id, weight=float(value))

# Load bad nodes
bad_nodes = set()
with open("bad.csv") as f:
    reader = csv.reader(f)
    next(reader) # skip header
    for row in reader:
        bad_nodes.add(row[0])

total_nodes = graph.number_of_nodes()
out_degrees = dict(graph.out_degree()) #out_degress of all nodes : nodes_value as key and oudegree as value
in_degrees = dict(graph.in_degree())
# print(in_degrees)

nodes = list(graph.nodes())  #list of nodes 

# Create a dictionary to map node labels to node IDs from 0 to total_nodes-1
node_id_map = {}
for i, node in enumerate(nodes):
    node_id_map[node] = i

# Transition matrix 
T = np.zeros((len(nodes), len(nodes)))
for p in nodes:
    for q in nodes:
        if(graph.has_edge(q, p)):
            T[node_id_map[str(p)]][node_id_map[str(q)]] = 1/out_degrees[str(q)]
        else:
            T[node_id_map[str(p)]][node_id_map[str(q)]] = 0

# Inverse Transion matrix (used in inverse page rank for selecting seed )
U = np.zeros((len(nodes), len(nodes)))
for p in nodes:
    for q in nodes:
        if(graph.has_edge(p,q)):
            T[node_id_map[str(p)]][node_id_map[str(q)]] = 1/in_degrees[str(q)]
        else:
            T[node_id_map[str(p)]][node_id_map[str(q)]] = 0

            
            
# computing d 
d = np.zeros(len(nodes))


# algorithm to select seed - either inverse page rank  or High Page rank ---- yet to implement


count = 0 
for i in nodes:
    if i in bad_nodes:
        d[node_id_map[str(i)]] = 0
    else:
        d[node_id_map[str(i)]] = 1
        count = count + 1

# Normalise d 
d = d/count
# print(d.sum())


# trust rank algo 
t = d 
num_iteration = 20 
alpha = 0.85
for i in range(num_iteration):
    t = alpha * np.dot(T,t) + (1-alpha)*d

    
trust_rank_scores = [] # to store the node and the corresponding trustrank score
for p in nodes :
    trust_rank_scores.append([p,t[node_id_map[str(p)]]])


# sort the scores in descending order
trust_rank_scores = sorted(trust_rank_scores, key=lambda x: x[1], reverse = True)
        
with open("final-rank.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(['node_id', 'trust_rank_scores'])
    for  i in  trust_rank_scores:
        writer.writerow([i[0], i[1]])

